### UCI Forest Type Dataset

- https://archive.ics.uci.edu/ml/datasets/covertype
- to verify if the results in the original paper can be reproduced
- used the same parameters as the tf v1 original implementation - https://github.com/google-research/google-research/tree/master/tabnet

In [2]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import LearningRateMonitor
from pytorch_lightning import seed_everything

from datasets import CovTypeDataModule
from tabnet_lightning import TabNetClassifier

In [3]:
seed_everything(1)
#path = "~/Nextcloud/projects/thesis/" # adjust path for data caching and logging
path = "../../../" # adjust path for data caching and logging

Global seed set to 1


In [4]:
dm = CovTypeDataModule(
    batch_size=16384,
    num_workers=8,
    cache_dir=path + "data/uci/covtype/",
    seed=0,  # use same seed / random state as tabnet original implementation
)

dm.prepare_data()
dm.setup()

In [5]:
classifier = TabNetClassifier(
    input_size=CovTypeDataModule.NUM_FEATURES,
    feature_size=128,
    decision_size=64,
    num_classes=CovTypeDataModule.NUM_LABELS,
    nr_layers=2,
    nr_shared_layers=2,
    nr_steps=5,
    gamma=1.5,

    # pytorch batch norm uses 1 - momentum (e.g. 0.3 pytorch is the same as tf 0.7)
    momentum=0.3,
    virtual_batch_size=512,

    lambda_sparse=0.0001,

    # define embeddings for categorical variables - otherwise raw value is used
    # categorical_indices=list(range(len(CovTypeDataModule.NUMERICAL_COLUMNS), CovTypeDataModule.NUM_FEATURES, 1)),
    # categorical_size=[2] * len(CovTypeDataModule.BINARY_COLUMNS),
    # embedding_dims=[1] * len(CovTypeDataModule.BINARY_COLUMNS),

    lr=0.02,
    optimizer="adam",
    scheduler="exponential_decay",
    scheduler_params={"decay_step": 500, "decay_rate": 0.95},

    #optimizer="adamw",
    #optimizer_params={"weight_decay": 0.0001},
    #scheduler="linear_with_warmup",
    #scheduler_params={"warmup_steps": 0.1},
)

/home/ck/anaconda3/envs/thesis/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [6]:
lr_monitor = LearningRateMonitor(logging_interval="step")
trainer = Trainer(default_root_dir=path + "logs/covtype/",

                  gpus=1,
                  # accelerator="none",

                  max_steps=2000,

                  fast_dev_run=False,
                  deterministic=True,

                  gradient_clip_algorithm="value",
                  gradient_clip_val=2000,

                  callbacks=[lr_monitor])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


In [7]:
trainer.fit(classifier, dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type             | Params
---------------------------------------------------
0 | encoder       | TabNet           | 470 K 
1 | classifier    | Linear           | 455   
2 | loss_fn       | CrossEntropyLoss | 0     
3 | train_metrics | MetricCollection | 0     
4 | val_metrics   | MetricCollection | 0     
5 | test_metrics  | MetricCollection | 0     
---------------------------------------------------
470 K     Trainable params
0         Non-trainable params
470 K     Total params
1.882     Total estimated model params size (MB)
/home/ck/anaconda3/envs/thesis/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

AttributeError: 'CovTypeDataModule' object has no attribute 'has_teardown_None'

In [19]:
trainer.test(classifier, datamodule=dm)


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test/AUROC': 0.9930207133293152,
 'test/Accuracy': 0.9102346897125244,
 'test/loss': 0.2228771448135376}
--------------------------------------------------------------------------------


[{'test/loss': 0.2228771448135376,
  'test/Accuracy': 0.9102346897125244,
  'test/AUROC': 0.9930207133293152}]